In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pickle
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_RandomForest
from metrics import ClassificationResult
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train_node1 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'


X_test_node1 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')

X_train=concat_motifs([X_train_node1, X_train_node2])
X_test=concat_motifs([X_test_node1, X_test_node2])
X_valid=concat_motifs([X_valid_node1, X_valid_node2])

In [3]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

In [4]:
X_train_pairs = X_train_normalized.reshape(X_train_normalized.shape[0],X_train_normalized.shape[2]*X_train_normalized.shape[3])
X_valid_pairs = X_valid_normalized.reshape(X_valid_normalized.shape[0],X_valid_normalized.shape[2]*X_valid_normalized.shape[3])
X_test_pairs = X_test_normalized.reshape(X_test_normalized.shape[0],X_test_normalized.shape[2]*X_test_normalized.shape[3])

In [5]:
X_train_pairs.shape

(6710, 3840)

# Random Forest

### with motifs only

# Estimating hyper-parameter using held out validation set

In [6]:
# X_train_valid = np.concatenate((X_train_normalized, X_valid_normalized), axis=0)
# y_train_valid = np.concatenate((y_train, y_valid), axis=0)
# test_fold to 0 for all samples that are part of the validation set, and to -1 for all other samples.
# valid_index=[-1 for i in range(X_train_normalized.shape[0])]+[0 for i in range(X_valid_normalized.shape[0])]
# param_grid = {'n_estimators': [100, 200, 500], 'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'log2' ], 'max_depth': [None, 2, 3, 5], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'min_impurity_split': [0.0 , 0.1, 0.2]}
# best_param={}
# rf = Genome3D_RandomForest(best_param)
# best_param = rf.train_cross_val(X_train_valid[:,:], [i for i in y_train_valid[:,0]], valid_index, param_grid)

# Random Forest Predictions

In [7]:
best_param = pickle.load(open('/users/mtaranov/genome3D/examples/from_sequence_features/nodes_features/best_param_rf.pkl', 'rb' ) )

In [8]:
rf = Genome3D_RandomForest(best_param)

In [9]:
rf.train(X_train_pairs, y_train)
preds_test = rf.predict(X_test_pairs)
preds_train = rf.predict(X_train_pairs)
print ('On Test:\n{}\n'.format(ClassificationResult(y_test, preds_test)))
print ('On Train:\n{}\n'.format(ClassificationResult(y_train, preds_train)))

/users/mtaranov/genome3D/models_3d.py:269: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.classifier.fit(X, y)


On Test:
Balanced Accuracy: 64.68%	 auROC: 0.671	 auPRC: 0.651	 auPRG: 0.349
Positve Accuracy: 69.43%	 Negative Accuracy: 59.92%
Recall at 5%|10%|20% FDR: 0.6%|2.8%|6.5%	 Num Positives: 494	 Num Negatives: 494	 

On Train:
Balanced Accuracy: 98.78%	 auROC: 0.999	 auPRC: 0.999	 auPRG: 0.999
Positve Accuracy: 99.52%	 Negative Accuracy: 98.03%
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 3355	 Num Negatives: 3355	 



# Random Forest Feature Selection

In [10]:
scores=rf.ImportanceSelect()

In [11]:
scores.shape

(3840,)

In [12]:
motif_names=np.loadtxt('/users/mtaranov/datasets_3d/dist_matched_PE/motifs/train_PE_out_btw_nodes_3_0.0001/motif_names.txt', dtype=str, delimiter='/')[:,6:].reshape(640,)
top_3_hits_names=[]
for i in motif_names:
    for j in range(3):
        top_3_hits_names.append(i)
top_3_hits_names = np.array(top_3_hits_names)

In [13]:
score_per_feature_node1=scores[0:5760:3]
score_per_feature_node2=scores[1:5761:3]
score_per_feature_window=scores[2:5762:3]

In [14]:
RF_Importance_Motifs_node1=top_3_hits_names[score_per_feature_node1.argsort()[::-1]]
RF_Importance_Motifs_node2=top_3_hits_names[score_per_feature_node2.argsort()[::-1]]
RF_Importance_Motifs_window=top_3_hits_names[score_per_feature_window.argsort()[::-1]]

In [15]:
print np.where(RF_Importance_Motifs_node1=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_node1=='CTCFL_HUMAN')

(array([], dtype=int64),)
(array([], dtype=int64),)


In [16]:
print np.where(RF_Importance_Motifs_node2=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_node2=='CTCFL_HUMAN')

(array([], dtype=int64),)
(array([], dtype=int64),)


In [17]:
print np.where(RF_Importance_Motifs_window=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_window=='CTCFL_HUMAN')

(array([], dtype=int64),)
(array([], dtype=int64),)


In [18]:
RF_Importance_Motifs_node1[:100]

array(['GATA1_HUMAN', 'CUX2_HUMAN', 'USF1_HUMAN', 'CREB1_HUMAN',
       'PKNX2_HUMAN', 'PKNX2_HUMAN', 'PURA_HUMAN', 'SP1_HUMAN',
       'TLX1_HUMAN', 'ELF2_HUMAN', 'AP2C_HUMAN', 'NFKB2_HUMAN',
       'BARH2_HUMAN', 'NFIA_HUMAN', 'PPARA_HUMAN', 'RARA_HUMAN',
       'PRRX2_HUMAN', 'FOXK1_HUMAN', 'ID4_HUMAN', 'NR1I2_HUMAN',
       'GATA1_HUMAN', 'RARA_HUMAN', 'MLX_HUMAN', 'NR2C1_HUMAN',
       'ETV6_HUMAN', 'NFYA_HUMAN', 'JUND_HUMAN', 'RUNX1_HUMAN',
       'HNF4G_HUMAN', 'SHOX2_HUMAN', 'LMX1B_HUMAN', 'RARB_HUMAN',
       'HME2_HUMAN', 'KLF6_HUMAN', 'E4F1_HUMAN', 'CPEB1_HUMAN',
       'HXD3_HUMAN', 'ELF2_HUMAN', 'CREM_HUMAN', 'SHOX_HUMAN',
       'MAFK_HUMAN', 'ONEC2_HUMAN', 'EVX1_HUMAN', 'PO3F2_HUMAN',
       'SNAI1_HUMAN', 'IRF7_HUMAN', 'PITX3_HUMAN', 'ETS2_HUMAN',
       'PHX2A_HUMAN', 'OVOL1_HUMAN', 'HTF4_HUMAN', 'EMX2_HUMAN',
       'IKZF1_HUMAN', 'FEV_HUMAN', 'GSC2_HUMAN', 'RUNX3_HUMAN',
       'CPEB1_HUMAN', 'JUNB_HUMAN', 'SOX18_HUMAN', 'LMX1B_HUMAN',
       'FOXH1_HUMAN', 'PO2F1_HU

In [19]:
RF_Importance_Motifs_node2[:100]

array(['RREB1_HUMAN', 'USF1_HUMAN', 'ZN639_HUMAN', 'KLF16_HUMAN',
       'NFIA_HUMAN', 'PKNX2_HUMAN', 'RXRG_HUMAN', 'PLAG1_HUMAN',
       'GCM2_HUMAN', 'MAFK_HUMAN', 'ZN350_HUMAN', 'TFDP1_HUMAN',
       'RHXF1_HUMAN', 'ZFX_HUMAN', 'LMX1B_HUMAN', 'RXRB_HUMAN',
       'CEBPZ_HUMAN', 'DBP_HUMAN', 'USF1_HUMAN', 'KLF8_HUMAN',
       'PKNX2_HUMAN', 'E2F8_HUMAN', 'OVOL1_HUMAN', 'CR3L2_HUMAN',
       'CREM_HUMAN', 'CPEB1_HUMAN', 'NFIA_HUMAN', 'CUX2_HUMAN',
       'SOX18_HUMAN', 'SOX2_HUMAN', 'MEIS1_HUMAN', 'HXD3_HUMAN',
       'PO3F2_HUMAN', 'GATA2_HUMAN', 'PO4F1_HUMAN', 'GSC2_HUMAN',
       'HXA7_HUMAN', 'NKX61_HUMAN', 'JUN_HUMAN', 'EMX2_HUMAN',
       'HXA13_HUMAN', 'SMAD4_HUMAN', 'NFIA_HUMAN', 'ZN589_HUMAN',
       'BHA15_HUMAN', 'PROP1_HUMAN', 'RXRB_HUMAN', 'ZN639_HUMAN',
       'NFYB_HUMAN', 'RARA_HUMAN', 'PPARA_HUMAN', 'SHOX_HUMAN',
       'MYBB_HUMAN', 'CEBPD_HUMAN', 'KLF14_HUMAN', 'GATA2_HUMAN',
       'PO6F1_HUMAN', 'HTF4_HUMAN', 'NR0B1_HUMAN', 'GCM2_HUMAN',
       'EMX2_HUMAN', 'FOXJ

In [20]:
RF_Importance_Motifs_window[:100]

array(['USF1_HUMAN', 'HEY1_HUMAN', 'FEV_HUMAN', 'MAFK_HUMAN',
       'OVOL1_HUMAN', 'NFYA_HUMAN', 'ETV2_HUMAN', 'PLAG1_HUMAN',
       'GBX1_HUMAN', 'TBX19_HUMAN', 'GATA1_HUMAN', 'ZFHX3_HUMAN',
       'PHX2A_HUMAN', 'SHOX2_HUMAN', 'BARH2_HUMAN', 'FOXD1_HUMAN',
       'LMX1B_HUMAN', 'CREB1_HUMAN', 'FUBP1_HUMAN', 'TBX20_HUMAN',
       'ID4_HUMAN', 'CUX2_HUMAN', 'SP1_HUMAN', 'ONEC3_HUMAN',
       'GATA1_HUMAN', 'RARA_HUMAN', 'IKZF1_HUMAN', 'PURA_HUMAN',
       'PKNX2_HUMAN', 'SOX18_HUMAN', 'NFIA_HUMAN', 'MYBB_HUMAN',
       'ZN639_HUMAN', 'PO4F1_HUMAN', 'KLF16_HUMAN', 'RUNX1_HUMAN',
       'RARA_HUMAN', 'EMX2_HUMAN', 'GSC2_HUMAN', 'USF1_HUMAN',
       'ELF2_HUMAN', 'OTX2_HUMAN', 'OLIG3_HUMAN', 'KLF15_HUMAN',
       'ATF2_HUMAN', 'MEF2C_HUMAN', 'HNF4G_HUMAN', 'MLX_HUMAN',
       'GSC2_HUMAN', 'E4F1_HUMAN', 'NFIA_HUMAN', 'BCL6_HUMAN',
       'MAFK_HUMAN', 'GATA2_HUMAN', 'FOS_HUMAN', 'HXD13_HUMAN',
       'LHX9_HUMAN', 'PO4F3_HUMAN', 'TFCP2_HUMAN', 'FOXI1_HUMAN',
       'FOXH1_HUMAN', 'REST_H

In [21]:
from sets import Set
uniq_RF_Importance_Motifs_node1 = Set(RF_Importance_Motifs_node1[:100])
uniq_RF_Importance_Motifs_node2 = Set(RF_Importance_Motifs_node2[:100])
uniq_RF_Importance_Motifs_window = Set(RF_Importance_Motifs_window[:100])


/users/mtaranov/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: the sets module is deprecated
  if __name__ == '__main__':


In [22]:
uniq_RF_Importance_Motifs_node1 & uniq_RF_Importance_Motifs_node2 & uniq_RF_Importance_Motifs_window


Set(['GSC2_HUMAN', 'NFIA_HUMAN', 'LMX1B_HUMAN', 'OVOL1_HUMAN', 'MAFK_HUMAN', 'E4F1_HUMAN', 'PKNX2_HUMAN', 'GATA1_HUMAN', 'BARH2_HUMAN', 'KLF16_HUMAN', 'PO4F3_HUMAN', 'MYBB_HUMAN', 'ELF2_HUMAN', 'KLF14_HUMAN', 'CPEB1_HUMAN', 'USF1_HUMAN', 'GLIS2_HUMAN', 'EMX2_HUMAN', 'HXD13_HUMAN', 'MEIS3_HUMAN', 'PO3F2_HUMAN', 'SHOX_HUMAN', 'PLAG1_HUMAN', 'ETV2_HUMAN', 'RARA_HUMAN', 'HXA13_HUMAN', 'FOXD1_HUMAN', 'SOX18_HUMAN', 'CUX2_HUMAN', 'SHOX2_HUMAN'])